In [17]:
import numpy as np
import pandas as pd 
import math
from scipy.fftpack import fft, fftshift, ifft
from scipy.fftpack import fftfreq
import pywt
import pywt.data 
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import roc_auc_score,classification_report,precision_score, recall_score

In [18]:
# # load data 
gtrain_data = pd.read_csv('../data/generative training data.csv')
gtest_data = pd.read_csv('../data/generative test data.csv')
gvalid_data = pd.read_csv('../data/generative valid data.csv')
gtest_data = pd.concat((gtest_data,gvalid_data))
gtrain_x = np.array(gtrain_data)[:,:-1] 
gtrain_y = gtrain_data.iloc[:,-1]
gtest_x = np.array(gtest_data)[:,:-1]
gtest_y = gtest_data.iloc[:,-1]


otrain_data = pd.read_csv('../data/original training data.csv')
otest_data = pd.read_csv('../data/original test data.csv')
ovalid_data = pd.read_csv('../data/original valid data.csv')
otest_data = pd.concat((otest_data,ovalid_data))
otrain_x = np.array(otrain_data)[:,:-1] 
otrain_y = otrain_data.iloc[:,-1]
otest_x = np.array(otest_data)[:,:-1]
otest_y = otest_data.iloc[:,-1] 


In [19]:
def time_feature_extractor(data, sample_point):
    """Extract time-domain features: mean value, root mean square, waveform factor, clearance factor"""
    means_list, rms_list, waveform_facs_list, clearance_facs_list = [], [], [], []
    for i in data:
        df_mean = pd.Series(i).mean()
        df_rms = math.sqrt(sum([x ** 2 for x in pd.Series(i)]) / sample_point)
        df_waveform_fac = df_rms / (abs(pd.Series(i)).mean())
        # Sum of squares
        sq_sum = 0
        for j in range(sample_point):
            sq_sum += math.sqrt(abs(i[j]))
        df_clearance_fac = (max(pd.Series(i))) / \
            pow((sq_sum / sample_point), 2)

        means_list.append(df_mean)
        rms_list.append(df_rms)
        waveform_facs_list.append(df_waveform_fac)
        clearance_facs_list.append(df_clearance_fac)

    return means_list, rms_list, waveform_facs_list, clearance_facs_list


def get_ps_array(data, num_fft):
    """Obtain power spectrum"""
    ps_list = []
    for i in data:
        # FFT
        Y = fft(i, num_fft)
        Y = np.abs(Y)
        # Reduce the DC component by N times, and the chord wave component by N
        # / 2 times
        Y[0] = Y[0] / num_fft
        Y[1:] = Y[1:] / (num_fft / 2)
        # power spectrum
        ps = Y ** 2 / num_fft
        ps = ps[:num_fft // 2]
        ps_list.append(ps)
    ps_array = np.array(ps_list)
    return ps_array


def frequency_features_extractor(ps_array, f):
    """obtain frequency domain features"""
    msf_list, variance_list = [], []
    for i in ps_array:
        # Mean square frequency
        msf = ((np.sqrt(i) * f).sum()) / (i.sum())
        # Power spectrum variance
        variance = i.var()

        msf_list.append(msf)
        variance_list.append(variance)

    return msf_list, variance_list


def wavelet_alternation(data):
    """extract wavelet feature"""
    sample_num, sample_dim = data.shape
    wp_features = np.zeros((sample_num, 8))
    for i in range(sample_num):
        single_data = data[i, :]
        # Wavelet transform to extract sample features
        wp = pywt.WaveletPacket(
            single_data,
            wavelet='db3',
            mode='symmetric',
            maxlevel=3)  
        # Get the node coefficient of level layer
        aaa = wp['aaa'].data
        aad = wp['aad'].data
        ada = wp['ada'].data
        add = wp['add'].data
        daa = wp['daa'].data
        dad = wp['dad'].data
        dda = wp['dda'].data
        ddd = wp['ddd'].data
        # Get norm of node
        ret1 = np.linalg.norm(aaa, ord=None)
        ret2 = np.linalg.norm(aad, ord=None)
        ret3 = np.linalg.norm(ada, ord=None)
        ret4 = np.linalg.norm(add, ord=None)
        ret5 = np.linalg.norm(daa, ord=None)
        ret6 = np.linalg.norm(dad, ord=None)
        ret7 = np.linalg.norm(dda, ord=None)
        ret8 = np.linalg.norm(ddd, ord=None)
        # obtain 8 nodes combined into eigenvector
        single_feature = [ret1, ret2, ret3, ret4, ret5, ret6, ret7, ret8]
        wp_features[i][:] = single_feature  # Array

    return wp_features


In [20]:
sample_point = 130
num_fft = 130 
fs = 10
df = fs / (num_fft)  # resolving power
# create frequency array
f = [df * n for n in range(0, num_fft)]
f = np.array(f[:num_fft // 2])

# obtain time domain feature list
gtrain_means_list, gtrain_rms_list, gtrain_waveform_facs_list, gtrain_clearance_facs_list = time_feature_extractor(gtrain_x, sample_point)
gtest_means_list, gtest_rms_list, gtest_waveform_facs_list, gtest_clearance_facs_list = time_feature_extractor(gtest_x, sample_point)

otrain_means_list, otrain_rms_list, otrain_waveform_facs_list, otrain_clearance_facs_list = time_feature_extractor(otrain_x, sample_point)
otest_means_list, otest_rms_list, otest_waveform_facs_list, otest_clearance_facs_list = time_feature_extractor(otest_x, sample_point)

# obtain frequency features
gtrain_ps = get_ps_array(gtrain_x, num_fft)
gtest_ps = get_ps_array(gtest_x, num_fft)
gtrain_msf_list, gtrain_variance_list = frequency_features_extractor(gtrain_ps, f)
gtest_msf_list, gtest_variance_list = frequency_features_extractor(gtest_ps, f)

otrain_ps = get_ps_array(otrain_x, num_fft)
otest_ps = get_ps_array(otest_x, num_fft)
otrain_msf_list, otrain_variance_list = frequency_features_extractor(otrain_ps, f)
otest_msf_list, otest_variance_list = frequency_features_extractor(otest_ps, f)

# obtain wavelet feature
gtrain_wp_data = wavelet_alternation(gtrain_x)
gtest_wp_data = wavelet_alternation(gtest_x)

otrain_wp_data = wavelet_alternation(otrain_x)
otest_wp_data = wavelet_alternation(otest_x) 

In [21]:
def create_df(wp_data,means_list,rms_list,waveform_facs_list,clearance_facs_list,msf_list,variance_list):
    data_x = pd.DataFrame(
        wp_data,
        columns=[
            'band 1 energy',
            'band 2 energy',
            'band 3 energy',
            'band 4 energy',
            'band 5 energy',
            'band 6 energy',
            'band 7 energy',
            'band 8 energy'])
    data_x['mean'] = means_list
    data_x['rms'] = rms_list
    data_x['waveform factor'] = waveform_facs_list
    data_x['clearance factor '] = clearance_facs_list
    data_x['msf'] = msf_list
    data_x['Power spectrum variance'] = variance_list
    
    return data_x


In [22]:
gtrain_x = create_df(gtrain_wp_data,gtrain_means_list,gtrain_rms_list,gtrain_waveform_facs_list,gtrain_clearance_facs_list,gtrain_msf_list,gtrain_variance_list)
gtest_x = create_df(gtest_wp_data,gtest_means_list,gtest_rms_list,gtest_waveform_facs_list,gtest_clearance_facs_list,gtest_msf_list,gtest_variance_list)

otrain_x = create_df(otrain_wp_data,otrain_means_list,otrain_rms_list,otrain_waveform_facs_list,otrain_clearance_facs_list,otrain_msf_list,otrain_variance_list)
otest_x = create_df(otest_wp_data,otest_means_list,otest_rms_list,otest_waveform_facs_list,otest_clearance_facs_list,otest_msf_list,otest_variance_list)

# 逻辑回归

In [28]:
transfer = StandardScaler()
x_train = transfer.fit_transform(gtrain_x)
x_test = transfer.transform(gtest_x)
y_train = gtrain_y 
y_test = gtest_y
model_log = LogisticRegression()
model_log.fit(x_train, y_train)
pre_y_log = model_log.predict(x_test)
# evaluate model 
print("训练集准确率",model_log.score(x_train, y_train))
print("测试集准确率",model_log.score(x_test, y_test))
y_pre = model_log.predict(x_test)
print(classification_report(y_test, y_pre,labels = (0,1),target_names=("正常","异常")))
print("逻辑回归模型在生成数据集的AUC指标：", roc_auc_score(y_test, y_pre))

训练集准确率 1.0
测试集准确率 1.0
              precision    recall  f1-score   support

          正常       1.00      1.00      1.00       342
          异常       1.00      1.00      1.00       366

    accuracy                           1.00       708
   macro avg       1.00      1.00      1.00       708
weighted avg       1.00      1.00      1.00       708

逻辑回归模型在生成数据集的AUC指标： 1.0


In [29]:
transfer = StandardScaler()
x_train = transfer.fit_transform(otrain_x)
x_test = transfer.transform(otest_x)
y_train = otrain_y
y_test = otest_y
model_log = LogisticRegression()
model_log.fit(x_train, y_train)
pre_y_log = model_log.predict(x_test)
# evaluate model 
print("训练集准确率",model_log.score(x_train, y_train))
print("测试集准确率",model_log.score(x_test, y_test))
y_pre = model_log.predict(x_test)
print(classification_report(y_test, y_pre,labels = (0,1),target_names=("正常","异常")))
print("逻辑回归模型在原数据集的AUC指标：", roc_auc_score(y_test, y_pre))

训练集准确率 1.0
测试集准确率 0.9944289693593314
              precision    recall  f1-score   support

          正常       0.99      1.00      1.00       354
          异常       1.00      0.60      0.75         5

    accuracy                           0.99       359
   macro avg       1.00      0.80      0.87       359
weighted avg       0.99      0.99      0.99       359

逻辑回归模型在原数据集的AUC指标： 0.8


# 决策树

In [27]:
x_train = otrain_x
x_test = otest_x
y_train = otrain_y
y_test = otest_y
model_tree = DecisionTreeClassifier()
model_tree.fit(x_train, y_train)
# evaluate model 
print("训练集准确率",model_tree.score(x_train, y_train))
print("测试集准确率",model_tree.score(x_test, y_test))
y_pre = model_tree.predict(x_test)
print(classification_report(y_test, y_pre,labels = (0,1),target_names=("正常","异常")))
print("决策树模型在原数据集的的AUC指标：", roc_auc_score(y_test, y_pre))

训练集准确率 1.0
测试集准确率 0.9916434540389972
              precision    recall  f1-score   support

          正常       0.99      1.00      1.00       354
          异常       0.75      0.60      0.67         5

    accuracy                           0.99       359
   macro avg       0.87      0.80      0.83       359
weighted avg       0.99      0.99      0.99       359

决策树模型在原数据集的的AUC指标： 0.798587570621469


In [108]:
x_train = gtrain_x
x_test = gtest_x
y_train = gtrain_y
y_test = gtest_y
model_tree = DecisionTreeClassifier()
model_tree.fit(x_train, y_train)
# evaluate model 
print("训练集准确率",model_tree.score(x_train, y_train))
print("测试集准确率",model_tree.score(x_test, y_test))
y_pre = model_tree.predict(x_test)
print(classification_report(y_test, y_pre,labels = (0,1),target_names=("正常","异常")))
print("决策树模型在生成数据集的的AUC指标：", roc_auc_score(y_test, y_pre))

训练集准确率 1.0
测试集准确率 1.0
              precision    recall  f1-score   support

          正常       1.00      1.00      1.00       342
          异常       1.00      1.00      1.00       366

    accuracy                           1.00       708
   macro avg       1.00      1.00      1.00       708
weighted avg       1.00      1.00      1.00       708

决策树模型在生成数据集的的AUC指标： 1.0


# SVM

In [30]:
x_train = otrain_x
x_test = otest_x
y_train = otrain_y
y_test = otest_y
model_svm = SVC(class_weight="balanced") 
model_svm.fit(x_train,y_train)
# evaluate model 
print("训练集准确率",model_svm.score(x_train, y_train))
print("测试集准确率",model_svm.score(x_test, y_test))
y_pre = model_svm.predict(x_test) 
print(classification_report(y_test, y_pre,labels = (0,1),target_names=("正常","异常")))
print("svm模型在原数据集的AUC指标：", roc_auc_score(y_test, y_pre))

训练集准确率 0.9888268156424581
测试集准确率 0.9916434540389972
              precision    recall  f1-score   support

          正常       0.99      1.00      1.00       354
          异常       0.75      0.60      0.67         5

    accuracy                           0.99       359
   macro avg       0.87      0.80      0.83       359
weighted avg       0.99      0.99      0.99       359

svm模型在原数据集的AUC指标： 0.798587570621469


In [152]:
x_train = gtrain_x
x_test = gtest_x
y_train = gtrain_y
y_test = gtest_y
model_svm = SVC(class_weight="balanced")
model_svm.fit(x_train,y_train)
# evaluate model 
print("训练集准确率",model_svm.score(x_train, y_train))
print("测试集准确率",model_svm.score(x_test, y_test))
y_pre = model_svm.predict(x_test) 
print(classification_report(y_test, y_pre,labels = (0,1),target_names=("正常","异常")))
print("svm模型在生成数据集的AUC指标：", roc_auc_score(y_test, y_pre))

训练集准确率 0.9198868991517436
测试集准确率 0.9011299435028248
              precision    recall  f1-score   support

          正常       0.84      0.98      0.91       342
          异常       0.98      0.83      0.90       366

    accuracy                           0.90       708
   macro avg       0.91      0.90      0.90       708
weighted avg       0.91      0.90      0.90       708

svm模型在生成数据集的AUC指标： 0.9037005081008532
